In [1]:
import asyncio

loop = asyncio.get_event_loop()

In [2]:
import unittest
from unittest import IsolatedAsyncioTestCase
from sdjwt.sdjwt import (
    generate_did_key_from_seed,
    create_w3c_vc_sd_jwt_for_data_attributes,
)
from sdjwt.didkey import DIDKey
from sdjwt.adapter import DataAttribute, DataAttributesAdapter





def create_w3c_vc_jwt_for_passport(didkey: DIDKey):
    credential_id = "urn:did:abc"
    credential_type = ["Passport"]
    credential_context = ["https://www.w3.org/2018/credentials/v1"]
    credential_schema = [
        {
            "id": "https://api-conformance.ebsi.eu/trusted-schemas-registry/v2/schemas/z3MgUFUkb722uq4x3dv5yAJmnNmzDFeK5UC8x83QoeLJM",
            "type": "FullJsonSchemaValidator2021",
        }
    ]
    data_attributes = [
        DataAttribute(name="section1.name", value="John Doe"),
        DataAttribute(name="section1.age", value=24),
        DataAttribute(name="section2.address.state", value="KL"),
        DataAttribute(name="section2.address.city", value="Kochi"),
    ]

    kid = "did:key:issuer_did#issuer_did"
    jti = credential_id
    iss = "did:key:issuer_did"
    sub = "did:key:datawallet_did"
    to_be_issued_credential = create_w3c_vc_sd_jwt_for_data_attributes(
        credential_id=credential_id,
        credential_type=credential_type,
        credential_context=credential_context,
        data_attributes=data_attributes,
        credential_status=None,
        terms_of_use=None,
        credential_schema=credential_schema,
        kid=kid,
        jti=jti,
        iss=iss,
        sub=sub,
        key=didkey.private_key,
        credential_issuer=didkey.generate()[0],
        limited_disclosure=True
    )

    return to_be_issued_credential

async def main():
    crypto_seed = "helloworld"
    key_did = await generate_did_key_from_seed(crypto_seed)
    key_did.generate()
    cred = create_w3c_vc_jwt_for_passport(didkey=key_did)
    print(cred)


await main()


eyJhbGciOiJFUzI1NiIsImtpZCI6ImRpZDprZXk6aXNzdWVyX2RpZCNpc3N1ZXJfZGlkIiwidHlwIjoiSldUIn0.eyJleHAiOjE3MTQ0Nzg0MjIsImlhdCI6MTcxNDQ3NDgyMiwiaXNzIjoiZGlkOmtleTppc3N1ZXJfZGlkIiwianRpIjoidXJuOmRpZDphYmMiLCJuYmYiOjE3MTQ0NzQ4MjIsInN1YiI6ImRpZDprZXk6ZGF0YXdhbGxldF9kaWQiLCJ2YyI6eyJAY29udGV4dCI6WyJodHRwczovL3d3dy53My5vcmcvMjAxOC9jcmVkZW50aWFscy92MSJdLCJjcmVkZW50aWFsU2NoZW1hIjpbeyJpZCI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvdHJ1c3RlZC1zY2hlbWFzLXJlZ2lzdHJ5L3YyL3NjaGVtYXMvejNNZ1VGVWtiNzIydXE0eDNkdjV5QUptbk5tekRGZUs1VUM4eDgzUW9lTEpNIiwidHlwZSI6IkZ1bGxKc29uU2NoZW1hVmFsaWRhdG9yMjAyMSJ9XSwiY3JlZGVudGlhbFN1YmplY3QiOnsiX3NkIjpbInJ5NWZTckVHaGlKaTRSZkEzdGQ5WmZXUE1mendoUGNfNVd4Z2xqTlZ4MjAiLCI1aTJZZ0dDSU5EckE2TVJqRVhxODFYSEJoZ2kxRXgwbnVBcFJBSEZCQXJBIl19LCJleHBpcmF0aW9uRGF0ZSI6IjIwMjQtMDQtMzBUMTI6MDA6MjJaIiwiaWQiOiJ1cm46ZGlkOmFiYyIsImlzc3VhbmNlRGF0ZSI6IjIwMjQtMDQtMzBUMTE6MDA6MjJaIiwiaXNzdWVkIjoiMjAyNC0wNC0zMFQxMTowMDoyMloiLCJpc3N1ZXIiOiJkaWQ6a2V5OnoyalF2SHVzVnZETHA4TGI1V1FuVUVva3R4a0VIZm9RMjkxRVc2WlBrSldH

In [3]:
def update_credential(credential, disclosure_mapping):
    def update_value(obj, path):
        for key in path[:-1]:
            obj = obj.setdefault(key, {})
        obj[path[-1]] = {"_sd": []}

    def iterate_mapping(obj, path):
        for key, value in obj.items():
            if isinstance(value, dict):
                new_path = path + [key]
                if "limitedDisclosure" in value and value["limitedDisclosure"]:
                    update_value(credential, new_path)
                iterate_mapping(value, new_path)

    iterate_mapping(disclosure_mapping, [])


credential = {
    "credentialSubject": {
        "Vehicle identifier": "1111",
        "section1": {"address": {"city": ""}, "occupation": ""},
        "section2": {
            "nestedfield": [
                {"value": "True"},
                {"value": "True"},
            ]
        },
    },
    "type": ["VerifiableCredential", "ParkingTicket"],
}

disclosure_mapping = {
    "credentialSubject": {
        "section1": {
            "address": {"limitedDisclosure": True},
            "occupation": {"limitedDisclosure": True},
        },
    }
}

update_credential(credential, disclosure_mapping)
print(credential)

{'credentialSubject': {'Vehicle identifier': '1111', 'section1': {'address': {'_sd': []}, 'occupation': {'_sd': []}}, 'section2': {'nestedfield': [{'value': 'True'}, {'value': 'True'}]}}, 'type': ['VerifiableCredential', 'ParkingTicket']}


In [2]:
from sdjwt.sdjwt import (
    generate_did_key_from_seed,
    create_w3c_vc_jwt_with_disclosure_mapping,
)
from sdjwt.didkey import DIDKey


def create_w3c_vc_jwt_for_driving_license(didkey: DIDKey):
    credential_id = "urn:did:abc"
    credential_type = ["DrivingLicense"]
    credential_context = ["https://www.w3.org/2018/credentials/v1"]
    credential_schema = [
        {
            "id": "https://api-conformance.ebsi.eu/trusted-schemas-registry/v2/schemas/z3MgUFUkb722uq4x3dv5yAJmnNmzDFeK5UC8x83QoeLJM",
            "type": "FullJsonSchemaValidator2021",
        }
    ]
    credential_subject = {
        "credentialSubject": {
            "name": "Jane Doe",
            "Vehicle number": "87698",
            "address": {
                "countryCode": "SE",
                "postCode": "418 78",
                "streetNo": "Gunnar Engellaus vag 8, 91 1B",
                "town": "Stockholm",
            },
        }
    }

    disclosure_mapping = {
        "credentialSubject": {
            "name": {"limitedDisclosure": True},
            "Vehicle number": {"limitedDisclosure": True},
        }
    }
    kid = "did:key:issuer_did#issuer_did"
    jti = credential_id
    iss = "did:key:issuer_did"
    sub = "did:key:datawallet_did"
    to_be_issued_credential = create_w3c_vc_jwt_with_disclosure_mapping(
        credential_id=credential_id,
        credential_type=credential_type,
        credential_context=credential_context,
        credential_subject=credential_subject,
        credential_status=None,
        terms_of_use=None,
        credential_schema=credential_schema,
        kid=kid,
        jti=jti,
        iss=iss,
        sub=sub,
        key=didkey.private_key,
        credential_issuer=didkey.generate()[0],
        disclosure_mapping=disclosure_mapping,
    )

    return to_be_issued_credential


async def main():
    crypto_seed = "helloworld"
    key_did = await generate_did_key_from_seed(crypto_seed)
    key_did.generate()
    cred = create_w3c_vc_jwt_for_driving_license(didkey=key_did)
    print(cred)
    # for disclosure in cred.split("~")[1:]:
    #     print(disclosure)


await main()

eyJhbGciOiJFUzI1NiIsImtpZCI6ImRpZDprZXk6aXNzdWVyX2RpZCNpc3N1ZXJfZGlkIiwidHlwIjoiSldUIn0.eyJleHAiOjE3MTQ1Njc2NjcsImlhdCI6MTcxNDU2NDA2NywiaXNzIjoiZGlkOmtleTppc3N1ZXJfZGlkIiwianRpIjoidXJuOmRpZDphYmMiLCJuYmYiOjE3MTQ1NjQwNjcsInN1YiI6ImRpZDprZXk6ZGF0YXdhbGxldF9kaWQiLCJ2YyI6eyJAY29udGV4dCI6WyJodHRwczovL3d3dy53My5vcmcvMjAxOC9jcmVkZW50aWFscy92MSJdLCJjcmVkZW50aWFsU2NoZW1hIjpbeyJpZCI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvdHJ1c3RlZC1zY2hlbWFzLXJlZ2lzdHJ5L3YyL3NjaGVtYXMvejNNZ1VGVWtiNzIydXE0eDNkdjV5QUptbk5tekRGZUs1VUM4eDgzUW9lTEpNIiwidHlwZSI6IkZ1bGxKc29uU2NoZW1hVmFsaWRhdG9yMjAyMSJ9XSwiY3JlZGVudGlhbFN1YmplY3QiOnsiX3NkIjpbImY2TGxsVGJSdENUeEVrVnpmd3pTYzZlVTU0WXN0d1A0NTN2UHhSc2hrVFkiLCJGY2g3VkpBT1M0YVowaTh4ZV9vUWppU0MzUy1RUTljX3BBRGxtOU05UWFNIl0sImFkZHJlc3MiOnsiY291bnRyeUNvZGUiOiJTRSIsInBvc3RDb2RlIjoiNDE4IDc4Iiwic3RyZWV0Tm8iOiJHdW5uYXIgRW5nZWxsYXVzIHZhZyA4LCA5MSAxQiIsInRvd24iOiJTdG9ja2hvbG0ifX0sImV4cGlyYXRpb25EYXRlIjoiMjAyNC0wNS0wMVQxMjo0Nzo0N1oiLCJpZCI6InVybjpkaWQ6YWJjIiwiaXNzdWFuY2VEYXRlIjoiMjAy

In [2]:
from sdjwt.pex import (
    validate_and_deserialise_presentation_definition,
    validate_and_deserialise_presentation_submission,
)


pd = validate_and_deserialise_presentation_definition(
    presentation_definition={
        "id": "67341729-cb5a-4d2c-a7e0-9e3472c203b9",
        "format": {"jwt_vc": {"alg": ["ES256"]}, "jwt_vp": {"alg": ["ES256"]}},
        "input_descriptors": [
            {
                "id": "430b7182-a237-4cb1-8d84-5163cafbfd36",
                "constraints": {
                    "fields": [
                        {
                            "path": ["$.type"],
                            "filter": {
                                "type": "array",
                                "contains": {"const": "VerifiablePortableDocumentA1"},
                            },
                        }
                    ]
                },
            }
        ],
    }
)
print(pd)

ps = validate_and_deserialise_presentation_submission(
    presentation_submission={
        "presentation_submission": {
            "definition_id": "841fd89b-f987-4052-88fc-30affccfd99c",
            "descriptor_map": [
                {
                    "format": "vc+sd-jwt",
                    "id": "2cfa0952-76ae-4172-a1c1-d223a8c00ce8",
                    "path": "$",
                    "path_nested": {
                        "format": "vc+sd-jwt",
                        "id": "2cfa0952-76ae-4172-a1c1-d223a8c00ce8",
                        "path": "$.verifiableCredential[0]",
                    },
                }
            ],
            "id": "f7a98fa3-23b8-4d01-a656-b7f9f95ee470",
        }
    }
)
print(ps)

id='67341729-cb5a-4d2c-a7e0-9e3472c203b9' input_descriptors=[InputDescriptor(id='430b7182-a237-4cb1-8d84-5163cafbfd36', name=None, purpose=None, constraints={'fields': [Field(id=None, optional=None, path=['$.type'], purpose=None, name=None, intent_to_retain=None, filter={'type': 'array', 'contains': {'const': 'VerifiablePortableDocumentA1'}})]})] format={'jwt_vc': {'alg': ['ES256']}, 'jwt_vp': {'alg': ['ES256']}}
presentation_submission=PresentationSubmission(id='f7a98fa3-23b8-4d01-a656-b7f9f95ee470', definition_id='841fd89b-f987-4052-88fc-30affccfd99c', descriptor_map=[Descriptor(id='2cfa0952-76ae-4172-a1c1-d223a8c00ce8', path='$', path_nested={'format': 'vc+sd-jwt', 'id': '2cfa0952-76ae-4172-a1c1-d223a8c00ce8', 'path': '$.verifiableCredential[0]'}, format='vc+sd-jwt')])


In [1]:
from sdjwt.pex import validate_vp_token_against_presentation_submission_and_presentation_definition

vp_token = "eyJraWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1BnWW95dHlrVVozZXlxaHQxajlLYnNLM0dkaHk3NWJ1eWtBVDlNSEc4S2NENEVUZUNQdGhhNVBtc1BmS3lZekZuM3loS0pBR1VFNHNKcE5KM0NtWW55SFRLZEFmYlZZdENqMk04ZGJmRm4xaVRSVk5xWXlRSlBTdFRGbzdUcGk0UEZXcEV4dnVaWWRNTERqMlh4V29IZzgjejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUtic0szR2RoeTc1YnV5a0FUOU1IRzhLY0Q0RVRlQ1B0aGE1UG1zUGZLeVl6Rm4zeWhLSkFHVUU0c0pwTkozQ21ZbnlIVEtkQWZiVll0Q2oyTThkYmZGbjFpVFJWTnFZeVFKUFN0VEZvN1RwaTRQRldwRXh2dVpZZE1MRGoyWHhXb0hnOCIsInR5cCI6IkpXVCIsImFsZyI6IkVTMjU2IiwiandrIjp7Imt0eSI6IkVDIiwiY3J2IjoiUC0yNTYiLCJ4Ijoib2VUUXNkVHByNy1yYTdrcEh2N2pPV1M2V19kdFFTTGlYN0VYbU1CVzJRRSIsInkiOiJJRUoyT3FXMW41SFdBVmJkaHlvT1k5alJpaXB0MVo5THZfamtHRVk5RW9NIn19.eyJhdWQiOiJodHRwczpcL1wvc3dlZGVuLWV1ZGktd2FsbGV0LmlncmFudC5pbyIsInN1YiI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUtic0szR2RoeTc1YnV5a0FUOU1IRzhLY0Q0RVRlQ1B0aGE1UG1zUGZLeVl6Rm4zeWhLSkFHVUU0c0pwTkozQ21ZbnlIVEtkQWZiVll0Q2oyTThkYmZGbjFpVFJWTnFZeVFKUFN0VEZvN1RwaTRQRldwRXh2dVpZZE1MRGoyWHhXb0hnOCIsIm5iZiI6MTcxNDQ5Njk2MSwiaXNzIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JzSzNHZGh5NzVidXlrQVQ5TUhHOEtjRDRFVGVDUHRoYTVQbXNQZkt5WXpGbjN5aEtKQUdVRTRzSnBOSjNDbVlueUhUS2RBZmJWWXRDajJNOGRiZkZuMWlUUlZOcVl5UUpQU3RURm83VHBpNFBGV3BFeHZ1WllkTUxEajJYeFdvSGc4IiwidnAiOnsiaG9sZGVyIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JzSzNHZGh5NzVidXlrQVQ5TUhHOEtjRDRFVGVDUHRoYTVQbXNQZkt5WXpGbjN5aEtKQUdVRTRzSnBOSjNDbVlueUhUS2RBZmJWWXRDajJNOGRiZkZuMWlUUlZOcVl5UUpQU3RURm83VHBpNFBGV3BFeHZ1WllkTUxEajJYeFdvSGc4IiwiaWQiOiJ1cm46dXVpZDphMzYwZDNjMS0xZjFhLTRmNTUtYTQ1Mi00OTcwOWJiNjU0NTQiLCJ0eXBlIjpbIlZlcmlmaWFibGVQcmVzZW50YXRpb24iXSwiQGNvbnRleHQiOlsiaHR0cHM6XC9cL3d3dy53My5vcmdcLzIwMThcL2NyZWRlbnRpYWxzXC92MSJdLCJ2ZXJpZmlhYmxlQ3JlZGVudGlhbCI6WyJleUpoYkdjaU9pSkZVekkxTmlJc0ltdHBaQ0k2SW1ScFpEcHJaWGs2ZWpKa2JYcEVPREZqWjFCNE9GWnJhVGRLWW5WMVRXMUdXWEpYVUdkWmIzbDBlV3RWV2pObGVYRm9kREZxT1V0aWMyTllZVEYxUjI5bGJubG1iVlY1V1V0a1RWQldRMmRDVWtwQmQwVkdhVWd5UTFscWRIbG1abXRNVERGelpGTlRXbTVRY2twdGIwZE9abTE2UzFoR04yMDJTMEZ4TVZkalpYVkxVbFpUU2tWeGNYVnZRa2g2UVdWcFptMXpUVzFHZG1GSWFWVmFjRWh5Um1ab2EyaHplR1puVWxabmNXUlNkamwzYVdFM1pEVlpPQ042TW1SdGVrUTRNV05uVUhnNFZtdHBOMHBpZFhWTmJVWlpjbGRRWjFsdmVYUjVhMVZhTTJWNWNXaDBNV281UzJKelkxaGhNWFZIYjJWdWVXWnRWWGxaUzJSTlVGWkRaMEpTU2tGM1JVWnBTREpEV1dwMGVXWm1hMHhNTVhOa1UxTmFibEJ5U20xdlIwNW1iWHBMV0VZM2JUWkxRWEV4VjJObGRVdFNWbE5LUlhGeGRXOUNTSHBCWldsbWJYTk5iVVoyWVVocFZWcHdTSEpHWm1ocmFITjRabWRTVm1keFpGSjJPWGRwWVRka05WazRJaXdpZEhsd0lqb2lTbGRVSW4wLmV5SmxlSEFpT2pFM01UUTFNREExTkRVc0ltbGhkQ0k2TVRjeE5EUTVOamswTlN3aWFYTnpJam9pWkdsa09tdGxlVHA2TW1SdGVrUTRNV05uVUhnNFZtdHBOMHBpZFhWTmJVWlpjbGRRWjFsdmVYUjVhMVZhTTJWNWNXaDBNV281UzJKelkxaGhNWFZIYjJWdWVXWnRWWGxaUzJSTlVGWkRaMEpTU2tGM1JVWnBTREpEV1dwMGVXWm1hMHhNTVhOa1UxTmFibEJ5U20xdlIwNW1iWHBMV0VZM2JUWkxRWEV4VjJObGRVdFNWbE5LUlhGeGRXOUNTSHBCWldsbWJYTk5iVVoyWVVocFZWcHdTSEpHWm1ocmFITjRabWRTVm1keFpGSjJPWGRwWVRka05WazRJaXdpYW5ScElqb2lkWEp1T21ScFpEb3hObUkzTVdVNE9DMDFZemc1TFRRNFpUTXRZV0pqTmkwMU0yUTRPV1V5T1RnME0yRWlMQ0p1WW1ZaU9qRTNNVFEwT1RZNU5EVXNJbk4xWWlJNklpSXNJblpqSWpwN0lrQmpiMjUwWlhoMElqcGJJbWgwZEhCek9pOHZkM2QzTG5jekxtOXlaeTh5TURFNEwyTnlaV1JsYm5ScFlXeHpMM1l4SWwwc0ltTnlaV1JsYm5ScFlXeFRZMmhsYldFaU9sdDdJbWxrSWpvaWFIUjBjSE02THk5aGNHa3RZMjl1Wm05eWJXRnVZMlV1WldKemFTNWxkUzkwY25WemRHVmtMWE5qYUdWdFlYTXRjbVZuYVhOMGNua3Zkakl2YzJOb1pXMWhjeTk2TTAxblZVWlZhMkkzTWpKMWNUUjRNMlIyTlhsQlNtMXVUbTE2UkVabFN6VlZRemg0T0ROUmIyVk1TazBpTENKMGVYQmxJam9pUm5Wc2JFcHpiMjVUWTJobGJXRldZV3hwWkdGMGIzSXlNREl4SW4xZExDSmpjbVZrWlc1MGFXRnNVM1ZpYW1WamRDSTZleUpwWkNJNmJuVnNiQ3dpYzJWamRHbHZiakVpT25zaVpHRjBaVUpwY25Sb0lqb2lNVGsyT0MweE1pMHlPU0lzSW1admNtVnVZVzFsY3lJNklrTm9ZWEpzYjNSMFpTSXNJbTVoZEdsdmJtRnNhWFJwWlhNaU9sc2lVMFVpWFN3aWNHVnljMjl1WVd4SlpHVnVkR2xtYVdOaGRHbHZiazUxYldKbGNpSTZJakU1TmpneE1qSTVMVEUwTVRJaUxDSndiR0ZqWlVKcGNuUm9JanA3SW1OdmRXNTBjbmxEYjJSbElqb2lVMFVpTENKeVpXZHBiMjRpT2lKVGRHOWphMmh2YkcwaUxDSjBiM2R1SWpvaVUzUnZZMnRvYjJ4dEluMHNJbk5sZUNJNklrWmxiV0ZzWlNJc0luTjBZWFJsVDJaU1pYTnBaR1Z1WTJWQlpHUnlaWE56SWpwN0ltTnZkVzUwY25sRGIyUmxJam9pVTBVaUxDSndiM04wUTI5a1pTSTZJalF4T0NBM09DSXNJbk4wY21WbGRFNXZJam9pUjNWdWJtRnlJRVZ1WjJWc2JHRjFjeUIyWVdjZ09Dd2dPVEVnTVVJaUxDSjBiM2R1SWpvaVUzUnZZMnRvYjJ4dEluMHNJbk4wWVhSbFQyWlRkR0Y1UVdSa2NtVnpjeUk2ZXlKamIzVnVkSEo1UTI5a1pTSTZJbE5GSWl3aWNHOXpkRU52WkdVaU9pSTBNVGdnTnpnaUxDSnpkSEpsWlhST2J5STZJa2QxYm01aGNpQkZibWRsYkd4aGRYTWdkbUZuSURnc0lEa3hJREZDSWl3aWRHOTNiaUk2SWxOMGIyTnJhRzlzYlNKOUxDSnpkWEp1WVcxbElqb2lRVzVrWlhKemIyNGlMQ0p6ZFhKdVlXMWxRWFJDYVhKMGFDSTZJa0Z1WkdWeWMyOXVJbjBzSW5ObFkzUnBiMjR5SWpwN0ltTmxjblJwWm1sallYUmxSbTl5UkhWeVlYUnBiMjVCWTNScGRtbDBlU0k2ZEhKMVpTd2laR1YwWlhKdGFXNWhkR2x2YmxCeWIzWnBjMmx2Ym1Gc0lqcG1ZV3h6WlN3aVpXNWthVzVuUkdGMFpTSTZJakl3TWpRdE1EY3RNRE1pTENKdFpXMWlaWEpUZEdGMFpWZG9hV05vVEdWbmFYTnNZWFJwYjI1QmNIQnNhV1Z6SWpvaVNWUWlMQ0p6ZEdGeWRHbHVaMFJoZEdVaU9pSXlNREl6TFRBNUxUSXhJaXdpZEhKaGJuTnBkR2x2YmxKMWJHVnpRWEJ3YkhsQmMwVkRPRGd6TWpBd05DSTZabUZzYzJWOUxDSnpaV04wYVc5dU15STZleUpqYVhacGJFRnVaRVZ0Y0d4dmVXVmtVMlZzWmtWdGNHeHZlV1ZrSWpwbVlXeHpaU3dpWTJsMmFXeFRaWEoyWVc1MElqcG1ZV3h6WlN3aVkyOXVkSEpoWTNSVGRHRm1aaUk2Wm1Gc2MyVXNJbVZ0Y0d4dmVXVmtRVzVrVTJWc1prVnRjR3h2ZVdWa0lqcG1ZV3h6WlN3aVpXMXdiRzk1WldSVWQyOVBjazF2Y21WVGRHRjBaWE1pT21aaGJITmxMQ0psZUdObGNIUnBiMjRpT21aaGJITmxMQ0psZUdObGNIUnBiMjVFWlhOamNtbHdkR2x2YmlJNklpSXNJbVpzYVdkb2RFTnlaWGROWlcxaVpYSWlPbVpoYkhObExDSnRZWEpwYm1WeUlqcG1ZV3h6WlN3aWNHOXpkR1ZrUlcxd2JHOTVaV1JRWlhKemIyNGlPbVpoYkhObExDSndiM04wWldSVFpXeG1SVzF3Ykc5NVpXUlFaWEp6YjI0aU9uUnlkV1VzSW5ObGJHWkZiWEJzYjNsbFpGUjNiMDl5VFc5eVpWTjBZWFJsY3lJNlptRnNjMlVzSW5kdmNtdHBibWRKYmxOMFlYUmxWVzVrWlhJeU1TSTZabUZzYzJWOUxDSnpaV04wYVc5dU5DSTZleUpsYlhCc2IzbGxaU0k2Wm1Gc2MyVXNJbVZ0Y0d4dmVXVnlVMlZzWmtWdGNHeHZlV1ZrUVdOMGFYWnBkSGxEYjJSbGN5STZXeUl4T0RnNU1URXpNalEwSWwwc0ltNWhiV1ZDZFhOcGJtVnpjMDVoYldVaU9pSldiMngyYnlJc0luSmxaMmx6ZEdWeVpXUkJaR1J5WlhOeklqcDdJbU52ZFc1MGNubERiMlJsSWpvaVUwVWlMQ0p3YjNOMFEyOWtaU0k2SWpReE9DQTNPQ0lzSW5OMGNtVmxkRTV2SWpvaVIzVnVibUZ5SUVWdVoyVnNiR0YxY3lCMlhIVXdNR1UwWnlBNExDQXhOalFnUVNJc0luUnZkMjRpT2lKSGIzUmxZbTl5WnlKOUxDSnpaV3htUlcxd2JHOTVaV1JCWTNScGRtbDBlU0k2ZEhKMVpYMHNJbk5sWTNScGIyNDFJanA3SW01dlJtbDRaV1JCWkdSeVpYTnpJanBtWVd4elpTd2lkMjl5YTFCc1lXTmxRV1JrY21WemMyVnpJanBiZXlKaFpHUnlaWE56SWpwN0ltTnZkVzUwY25sRGIyUmxJam9pU1ZRaUxDSndiM04wUTI5a1pTSTZJak0wTVRNeUlpd2ljM1J5WldWMFRtOGlPaUpRYVdGNmVtRWdSSFZqWVNCa1pXZHNhU0JCWW5KMWVucHBJRElzSURRME1DSXNJblJ2ZDI0aU9pSlVjbWxsYzNSbEluMHNJbk5sY1c1dklqb3hmVjBzSW5kdmNtdFFiR0ZqWlU1aGJXVnpJanBiZXlKamIyMXdZVzU1VG1GdFpWWmxjM05sYkU1aGJXVWlPaUpCYzNOcFkzVnlZWHBwYjI1cElFZGxibVZ5WVd4cElGTXVjQzVCSWl3aWMyVnhibThpT2pGOVhYMHNJbk5sWTNScGIyNDJJanA3SW1Ga1pISmxjM01pT25zaVkyOTFiblJ5ZVVOdlpHVWlPaUpDUlNJc0luQnZjM1JEYjJSbElqb2lNVEF3TUNJc0luTjBjbVZsZEU1dklqb2lUV0ZwYmlCVGRISmxaWFFnTVNJc0luUnZkMjRpT2lKQ2NuVnpjMlZzY3lKOUxDSmtZWFJsSWpvaU1qQXlNeTB3T1Mwd055SXNJbVZ0WVdsc0lqb2lhVzVtYjBCdWMzTnBMV0psTG1WMUlpd2lhVzV6ZEdsMGRYUnBiMjVKUkNJNklrNVRVMGt0UWtVdE1ERWlMQ0p1WVcxbElqb2lUbUYwYVc5dVlXd2dVMjlqYVdGc0lGTmxZM1Z5YVhSNUlFOW1abWxqWlNJc0ltOW1abWxqWlVaaGVFNXZJam9pTURnd01DQTVPRGMyTlNJc0ltOW1abWxqWlZCb2IyNWxUbThpT2lJd09EQXdJREV5TXpRMUlpd2ljMmxuYm1GMGRYSmxJam9pVDJabWFXTnBZV3dnYzJsbmJtRjBkWEpsSW4xOUxDSmxlSEJwY21GMGFXOXVSR0YwWlNJNklqSXdNalF0TURRdE16QlVNVGc2TURrNk1EVmFJaXdpYVdRaU9pSjFjbTQ2Wkdsa09qRTJZamN4WlRnNExUVmpPRGt0TkRobE15MWhZbU0yTFRVelpEZzVaVEk1T0RRellTSXNJbWx6YzNWaGJtTmxSR0YwWlNJNklqSXdNalF0TURRdE16QlVNVGM2TURrNk1EVmFJaXdpYVhOemRXVmtJam9pTWpBeU5DMHdOQzB6TUZReE56b3dPVG93TlZvaUxDSnBjM04xWlhJaU9pSmthV1E2YTJWNU9ub3laRzE2UkRneFkyZFFlRGhXYTJrM1NtSjFkVTF0UmxseVYxQm5XVzk1ZEhsclZWb3paWGx4YUhReGFqbExZbk5qV0dFeGRVZHZaVzU1Wm0xVmVWbExaRTFRVmtOblFsSktRWGRGUm1sSU1rTlphblI1Wm1aclRFd3hjMlJUVTFwdVVISktiVzlIVG1adGVrdFlSamR0Tmt0QmNURlhZMlYxUzFKV1UwcEZjWEYxYjBKSWVrRmxhV1p0YzAxdFJuWmhTR2xWV25CSWNrWm1hR3RvYzNobVoxSldaM0ZrVW5ZNWQybGhOMlExV1RnaUxDSjBlWEJsSWpwYklsWmxjbWxtYVdGaWJHVkRjbVZrWlc1MGFXRnNJaXdpVm1WeWFXWnBZV0pzWlVGMGRHVnpkR0YwYVc5dUlpd2lWbVZ5YVdacFlXSnNaVkJ2Y25SaFlteGxSRzlqZFcxbGJuUkJNU0pkTENKMllXeHBaRVp5YjIwaU9pSXlNREkwTFRBMExUTXdWREUzT2pBNU9qQTFXaUo5ZlEuQ014dEVQZUs3ZWJmNGthT0lOcFdGdzdZaWliY0haTW1pQUsxckZNNnZiWlN3U1BLTXBROTRQRkhpQUlXU3RXQk94OTdrMjNVRG9iOC1oOWdMbV9aZ1EiXX0sImV4cCI6MTcxNDQ5NzU2MSwiaWF0IjoxNzE0NDk2OTYxLCJub25jZSI6IjNmMzI2ZWU1LTQ5MjYtNDRlYi04NmNlLTU3Y2Y3YzlhZjUwMCIsImp0aSI6InVybjp1dWlkOmEzNjBkM2MxLTFmMWEtNGY1NS1hNDUyLTQ5NzA5YmI2NTQ1NCJ9.Ec0lfUv60aBZF1Ei_yXBwd1dcFEqOPnxqYzw5h73RsSJYipao6uosLMTcyLEMGAy4fJykd_q65nJURiHXSHQ-g"
validate_vp_token_against_presentation_submission_and_presentation_definition(vp_token)

VpTokenExpiredError: VP token expired

In [ ]:
req = {
  "issuanceMode": "InTime",
  "disclosureMapping": {
    "credentialSubject": {
      "name": {
        "limitedDisclosure": true
      },
      "Vehicle number": {
        "limitedDisclosure": true
      }
    }
  },
  "credential": {
    "type": [
      "DrivingLicense"
    ],
    "credentialSubject": {
      "name": "Jane Doe",
      "Vehicle number": "87698",
      "address": {
        "countryCode": "SE",
        "postCode": "418 78",
        "streetNo": "Gunnar Engellaus vag 8, 91 1B",
        "town": "Stockholm"
      }
    }
  },
  "userPin": ""
}




In [4]:
from jsonpath_ng import parse

data = {
    "name": "Jane Doe",
    "Vehicle number": "87698",
    "address": {
        "countryCode": "SE",
        "postCode": "418 78",
        "streetNo": "Gunnar Engellaus vag 8, 91 1B",
        "town": "Stockholm",
    },
}

jp = parse("'Vehicle number'")
jp.find(data)

[DatumInContext(value='87698', path=Fields('Vehicle number'), context=DatumInContext(value={'name': 'Jane Doe', 'Vehicle number': '87698', 'address': {'countryCode': 'SE', 'postCode': '418 78', 'streetNo': 'Gunnar Engellaus vag 8, 91 1B', 'town': 'Stockholm'}}, path=This(), context=None))]

In [2]:
vp = "eyJraWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1BnWW95dHlrVVozZXlxaHQxajlLYnFXeHFqc3R3eGJkb3ZENlBrbnB3VUFxSmZ0SGZUUmJzWVVmNVVlaWcyNm5abkJDcTF6SllhenVydkNiaVpjc0FWNDFzMUdGeTdmdG5zc0VWUFcydnZGZEFqNFlVSlZLQU52UHY2dFRSTVRvV2R5U01LTlJEc2JHWGU1ZXhSeHg1YWsjejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticVd4cWpzdHd4YmRvdkQ2UGtucHdVQXFKZnRIZlRSYnNZVWY1VWVpZzI2blpuQkNxMXpKWWF6dXJ2Q2JpWmNzQVY0MXMxR0Z5N2Z0bnNzRVZQVzJ2dkZkQWo0WVVKVktBTnZQdjZ0VFJNVG9XZHlTTUtOUkRzYkdYZTVleFJ4eDVhayIsInR5cCI6IkpXVCIsImFsZyI6IkVTMjU2IiwiandrIjp7Imt0eSI6IkVDIiwiY3J2IjoiUC0yNTYiLCJ4IjoiWGdaWS1KTzR2SGRVaXdNc1ZxQVVheWdUVGhta3E5cmxmdEhOMzdYT1lGWSIsInkiOiJrRkZPR3RQMEZadEwyc1BnZHc0alBhZGpkZzBCeUFvaUtwSTNtd2ZvLVZ3In19.eyJhdWQiOiJodHRwczpcL1wvb2lkNHZjLmlncmFudC5pbyIsInN1YiI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticVd4cWpzdHd4YmRvdkQ2UGtucHdVQXFKZnRIZlRSYnNZVWY1VWVpZzI2blpuQkNxMXpKWWF6dXJ2Q2JpWmNzQVY0MXMxR0Z5N2Z0bnNzRVZQVzJ2dkZkQWo0WVVKVktBTnZQdjZ0VFJNVG9XZHlTTUtOUkRzYkdYZTVleFJ4eDVhayIsIm5iZiI6MTcxNDYzNTQ4MCwiaXNzIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JxV3hxanN0d3hiZG92RDZQa25wd1VBcUpmdEhmVFJic1lVZjVVZWlnMjZuWm5CQ3ExekpZYXp1cnZDYmlaY3NBVjQxczFHRnk3ZnRuc3NFVlBXMnZ2RmRBajRZVUpWS0FOdlB2NnRUUk1Ub1dkeVNNS05SRHNiR1hlNWV4Unh4NWFrIiwidnAiOnsiaG9sZGVyIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JxV3hxanN0d3hiZG92RDZQa25wd1VBcUpmdEhmVFJic1lVZjVVZWlnMjZuWm5CQ3ExekpZYXp1cnZDYmlaY3NBVjQxczFHRnk3ZnRuc3NFVlBXMnZ2RmRBajRZVUpWS0FOdlB2NnRUUk1Ub1dkeVNNS05SRHNiR1hlNWV4Unh4NWFrIiwiaWQiOiJ1cm46dXVpZDoyMjNlZDYzMS0yNzVjLTQzNTgtOWJhYS1jNmRiNTkzZjBjYzciLCJ0eXBlIjpbIlZlcmlmaWFibGVQcmVzZW50YXRpb24iXSwiQGNvbnRleHQiOlsiaHR0cHM6XC9cL3d3dy53My5vcmdcLzIwMThcL2NyZWRlbnRpYWxzXC92MSJdLCJ2ZXJpZmlhYmxlQ3JlZGVudGlhbCI6WyJleUpoYkdjaU9pSkZVekkxTmlJc0ltdHBaQ0k2SW1ScFpEcHJaWGs2ZWpKa2JYcEVPREZqWjFCNE9GWnJhVGRLWW5WMVRXMUdXWEpYVUdkWmIzbDBlV3RWV2pObGVYRm9kREZxT1V0aWJtMXlkMWwzVEhOQ2QzcE1aRzVDT0hWU1FWSkNkbkoyUWtvMVVtRkxVMGhtT1V4WGRWUlJkbUZVVjBWcFZVRnZORWhyWVZWVmNUbHFZamxRVW0xNFJFMXJNa2RCWm0xU2VYSTBSM3BxTTFkQlRVZExNV2xtZUdSclJqWkxkM1pJZW1SYVoxQTBkRFp3YlhVNVZtbDZha05HUTBoNlVXSnZjbmxsZFZKMlprZDRheU42TW1SdGVrUTRNV05uVUhnNFZtdHBOMHBpZFhWTmJVWlpjbGRRWjFsdmVYUjVhMVZhTTJWNWNXaDBNV281UzJKdWJYSjNXWGRNYzBKM2VreGtia0k0ZFZKQlVrSjJjblpDU2pWU1lVdFRTR1k1VEZkMVZGRjJZVlJYUldsVlFXODBTR3RoVlZWeE9XcGlPVkJTYlhoRVRXc3lSMEZtYlZKNWNqUkhlbW96VjBGTlIwc3hhV1o0Wkd0R05rdDNka2g2WkZwblVEUjBObkJ0ZFRsV2FYcHFRMFpEU0hwUlltOXllV1YxVW5abVIzaHJJaXdpZEhsd0lqb2lTbGRVSW4wLmV5SmxlSEFpT2pFM01UUTJNRFExTmpVc0ltbGhkQ0k2TVRjeE5EWXdNRGsyTlN3aWFYTnpJam9pWkdsa09tdGxlVHA2TW1SdGVrUTRNV05uVUhnNFZtdHBOMHBpZFhWTmJVWlpjbGRRWjFsdmVYUjVhMVZhTTJWNWNXaDBNV281UzJKdWJYSjNXWGRNYzBKM2VreGtia0k0ZFZKQlVrSjJjblpDU2pWU1lVdFRTR1k1VEZkMVZGRjJZVlJYUldsVlFXODBTR3RoVlZWeE9XcGlPVkJTYlhoRVRXc3lSMEZtYlZKNWNqUkhlbW96VjBGTlIwc3hhV1o0Wkd0R05rdDNka2g2WkZwblVEUjBObkJ0ZFRsV2FYcHFRMFpEU0hwUlltOXllV1YxVW5abVIzaHJJaXdpYW5ScElqb2lkWEp1T21ScFpEbzJaR05pTXpkak5TMHhOMlJrTFRReU5ETXRPR0U0TXkxaFpqSmlabVF5TXprMk9ETWlMQ0p1WW1ZaU9qRTNNVFEyTURBNU5qVXNJbk4xWWlJNkltUnBaRHByWlhrNmVqSmtiWHBFT0RGaloxQjRPRlpyYVRkS1luVjFUVzFHV1hKWFVHZFpiM2wwZVd0VldqTmxlWEZvZERGcU9VdGljbE5HYnpKNlVFWm9Samh6YzNKWlUxcHdSV2RCWlhZMk1rMTJiVlo1WjNwTU4zcG1Oa1pWVTBSV2MwZHFRMEp0UW5sV1pIUjZVamt4WlVkM2QyVm9VM1ZaT0VoRVMzZGliMjkyV25ORGJXODJlVzgzVTJVNFkwTnZOMmR5Vm5OamRtMXpVMUpxUWpNMFp6aDZTMEpuTmxsTmNuaDNNMVpOWVdoRGVFaFlhbUZaWXlJc0luWmpJanA3SWtCamIyNTBaWGgwSWpwYkltaDBkSEJ6T2k4dmQzZDNMbmN6TG05eVp5OHlNREU0TDJOeVpXUmxiblJwWVd4ekwzWXhJbDBzSW1OeVpXUmxiblJwWVd4VFkyaGxiV0VpT2x0N0ltbGtJam9pYUhSMGNITTZMeTloY0drdFkyOXVabTl5YldGdVkyVXVaV0p6YVM1bGRTOTBjblZ6ZEdWa0xYTmphR1Z0WVhNdGNtVm5hWE4wY25rdmRqSXZjMk5vWlcxaGN5OTZNMDFuVlVaVmEySTNNakoxY1RSNE0yUjJOWGxCU20xdVRtMTZSRVpsU3pWVlF6aDRPRE5SYjJWTVNrMGlMQ0owZVhCbElqb2lSblZzYkVwemIyNVRZMmhsYldGV1lXeHBaR0YwYjNJeU1ESXhJbjFkTENKamNtVmtaVzUwYVdGc1UzVmlhbVZqZENJNmV5SmZjMlFpT2xzaU1rSTROMVpyYVUxSVVtTjZUMUoyUWxjMlJFRkNiMGhvVDJORE5XUTBUemt0TW1kSk4ycDBUbEZtZHlJc0lrOUpVMnBqVFZOTGRsZ3dPVkl5VkdGVU5tRm1TbUZKZFZReE5reGFhVmd6YUc1MGVWcGlUR0ZNU2pRaUxDSm5XRkpwTkZGNk5tZHpla1JNY0ZWUFJtSkVNMDVLVVRCQllsWktaV05pYmpoRVNuQjVVVWQ1TUVWUklsMHNJbWxrSWpvaVpHbGtPbXRsZVRwNk1tUnRla1E0TVdOblVIZzRWbXRwTjBwaWRYVk5iVVpaY2xkUVoxbHZlWFI1YTFWYU0yVjVjV2gwTVdvNVMySnlVMFp2TW5wUVJtaEdPSE56Y2xsVFduQkZaMEZsZGpZeVRYWnRWbmxuZWt3M2VtWTJSbFZUUkZaelIycERRbTFDZVZaa2RIcFNPVEZsUjNkM1pXaFRkVms0U0VSTGQySnZiM1phYzBOdGJ6WjViemRUWlRoalEyODNaM0pXYzJOMmJYTlRVbXBDTXpSbk9IcExRbWMyV1UxeWVIY3pWazFoYUVONFNGaHFZVmxqSWl3aWMyVmpkR2x2YmpFaU9uc2lYM05rSWpwYklrb3hWRGRqU1RkNmVrVlBORk5sY0ZvM1ZuVTJSbkp4WDBSaFNIZGhWREZWVGxaRlowNXVMV04xTjJNaUxDSkRRV3R6UWpNMWNUWXdja1Z6WlVacE1qUlFabWd5ZVZkSmVYTjFMWEJLY201NlFVdE5SRW94VVhGaklsMHNJbVJoZEdWQ2FYSjBhQ0k2SWpFNU5qZ3RNVEl0TWpraUxDSm1iM0psYm1GdFpYTWlPaUpEYUdGeWJHOTBkR1VpTENKd1pYSnpiMjVoYkVsa1pXNTBhV1pwWTJGMGFXOXVUblZ0WW1WeUlqb2lNVGsyT0RFeU1qa3RNVFF4TWlJc0luQnNZV05sUW1seWRHZ2lPbnNpWTI5MWJuUnllVU52WkdVaU9pSlRSU0lzSW5KbFoybHZiaUk2SWxOMGIyTnJhRzlzYlNJc0luUnZkMjRpT2lKVGRHOWphMmh2YkcwaWZTd2ljMlY0SWpvaVJtVnRZV3hsSWl3aWMzUmhkR1ZQWmxOMFlYbEJaR1J5WlhOeklqcDdJbU52ZFc1MGNubERiMlJsSWpvaVUwVWlMQ0p3YjNOMFEyOWtaU0k2SWpReE9DQTNPQ0lzSW5OMGNtVmxkRTV2SWpvaVIzVnVibUZ5SUVWdVoyVnNiR0YxY3lCMllXY2dPQ3dnT1RFZ01VSWlMQ0owYjNkdUlqb2lVM1J2WTJ0b2IyeHRJbjBzSW5OMWNtNWhiV1VpT2lKQmJtUmxjbk52YmlJc0luTjFjbTVoYldWQmRFSnBjblJvSWpvaVFXNWtaWEp6YjI0aWZTd2ljMlZqZEdsdmJqUWlPbnNpWDNOa0lqcGJJakJmZFZWV09YSlVVa2x2Ym1sblgyaHhlbGRJU2tWc1ZYQmpSek5DV1ZsRloyZG9ZMFEwVERZek9IY2lYU3dpYm1GdFpVSjFjMmx1WlhOelRtRnRaU0k2SWxadmJIWnZJaXdpY21WbmFYTjBaWEpsWkVGa1pISmxjM01pT25zaVgzTmtJanBiSWxCSFRWQmhjbTVmV1d4UFFYaFhObDk2VEZFeGJHbElNSEF3TmtKS1p5MVVjbWxGY25kR2MxbzVValFpTENKelZqQjJZbTQ1UlRKa1VERk9OMnBvWTNKS2VtaE5RWHBWTFRjd1FWVmpjVjltYzBrMFEwNUNXakk0SWwwc0ltTnZkVzUwY25sRGIyUmxJam9pVTBVaUxDSjBiM2R1SWpvaVIyOTBaV0p2Y21jaWZYMTlMQ0psZUhCcGNtRjBhVzl1UkdGMFpTSTZJakl3TWpRdE1EVXRNREZVTWpNNk1ESTZORFZhSWl3aWFXUWlPaUoxY200NlpHbGtPalprWTJJek4yTTFMVEUzWkdRdE5ESTBNeTA0WVRnekxXRm1NbUptWkRJek9UWTRNeUlzSW1semMzVmhibU5sUkdGMFpTSTZJakl3TWpRdE1EVXRNREZVTWpJNk1ESTZORFZhSWl3aWFYTnpkV1ZrSWpvaU1qQXlOQzB3TlMwd01WUXlNam93TWpvME5Wb2lMQ0pwYzNOMVpYSWlPaUprYVdRNmEyVjVPbm95WkcxNlJEZ3hZMmRRZURoV2EyazNTbUoxZFUxdFJsbHlWMUJuV1c5NWRIbHJWVm96WlhseGFIUXhhamxMWW01dGNuZFpkMHh6UW5kNlRHUnVRamgxVWtGU1FuWnlka0pLTlZKaFMxTklaamxNVjNWVVVYWmhWRmRGYVZWQmJ6UklhMkZWVlhFNWFtSTVVRkp0ZUVSTmF6SkhRV1p0VW5seU5FZDZhak5YUVUxSFN6RnBabmhrYTBZMlMzZDJTSHBrV21kUU5IUTJjRzExT1ZacGVtcERSa05JZWxGaWIzSjVaWFZTZG1aSGVHc2lMQ0owZVhCbElqcGJJbEJ2Y25SaFlteGxSRzlqZFcxbGJuUkJNU0pkTENKMllXeHBaRVp5YjIwaU9pSXlNREkwTFRBMUxUQXhWREl5T2pBeU9qUTFXaUo5ZlEublJPbDFreUhnOWtaSGpZcEJEb2JkcUxzdk1GSlZiT2IzZF9reGZnMWRKa1hBQlZleGdzYUpkRFExalBLY01FXy1oN0VGbzc5UThtUTgtZHVyekRmR3d-V3lKbE5qRmpNV0UxT0RVeU1URXlZak5tTXpnd05UaG1PRFprTkdZMU1EQXdOR1kyWkRjek1EZGtaV1ZpWXpOaVlqVmxabVF6WTJaa01tTTROek0wT0RJMUlpd2ljMlZqZEdsdmJqWWlMSHNpWVdSa2NtVnpjeUk2ZXlKamIzVnVkSEo1UTI5a1pTSTZJa0pGSWl3aWNHOXpkRU52WkdVaU9pSXhNREF3SWl3aWMzUnlaV1YwVG04aU9pSk5ZV2x1SUZOMGNtVmxkQ0F4SWl3aWRHOTNiaUk2SWtKeWRYTnpaV3h6SW4wc0ltUmhkR1VpT2lJeU1ESXpMVEE1TFRBM0lpd2laVzFoYVd3aU9pSnBibVp2UUc1emMya3RZbVV1WlhVaUxDSnBibk4wYVhSMWRHbHZia2xFSWpvaVRsTlRTUzFDUlMwd01TSXNJbTVoYldVaU9pSk9ZWFJwYjI1aGJDQlRiMk5wWVd3Z1UyVmpkWEpwZEhrZ1QyWm1hV05sSWl3aWIyWm1hV05sUm1GNFRtOGlPaUl3T0RBd0lEazROelkxSWl3aWIyWm1hV05sVUdodmJtVk9ieUk2SWpBNE1EQWdNVEl6TkRVaUxDSnphV2R1WVhSMWNtVWlPaUpQWm1acFkybGhiQ0J6YVdkdVlYUjFjbVVpZlYwIl19LCJleHAiOjE3MTQ2MzYwODAsImlhdCI6MTcxNDYzNTQ4MCwibm9uY2UiOiIyYjJiYWFhZS1jOTdkLTQzZGYtOTliYi0yYWJjNDQxYjFjMGIiLCJqdGkiOiJ1cm46dXVpZDoyMjNlZDYzMS0yNzVjLTQzNTgtOWJhYS1jNmRiNTkzZjBjYzcifQ.DLCXEONk8qd5Wyd2xQYcQje-ZQSVADdS4iJoJgRcq1BFS12qfGV0MzRvWt88Uwkw9CmnizALYDMymVbn7EDEVQ"
from sdjwt.pex import decode_header_and_claims_in_jwt

import base64
print(base64.urlsafe_b64decode(vp))
claims = decode_header_and_claims_in_jwt(vp)
# print(header)
print(claims)

Q
eyJhdWQiOiJodHRwczpcL1wvb2lkNHZjLmlncmFudC5pbyIsInN1YiI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticVd4cWpzdHd4YmRvdkQ2UGtucHdVQXFKZnRIZlRSYnNZVWY1VWVpZzI2blpuQkNxMXpKWWF6dXJ2Q2JpWmNzQVY0MXMxR0Z5N2Z0bnNzRVZQVzJ2dkZkQWo0WVVKVktBTnZQdjZ0VFJNVG9XZHlTTUtOUkRzYkdYZTVleFJ4eDVhayIsIm5iZiI6MTcxNDYzNTQ4MCwiaXNzIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JxV3hxanN0d3hiZG92RDZQa25wd1VBcUpmdEhmVFJic1lVZjVVZWlnMjZuWm5CQ3ExekpZYXp1cnZDYmlaY3NBVjQxczFHRnk3ZnRuc3NFVlBXMnZ2RmRBajRZVUpWS0FOdlB2NnRUUk1Ub1dkeVNNS05SRHNiR1hlNWV4Unh4NWFrIiwidnAiOnsiaG9sZGVyIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JxV3hxanN0d3hiZG92RDZQa25wd1VBcUpmdEhmVFJic1lVZjVVZWlnMjZuWm5CQ3ExekpZYXp1cnZDYmlaY3NBVjQxczFHRnk3ZnRuc3NFVlBXMnZ2RmRBajRZVUpWS0FOdlB2NnRUUk1Ub1dkeVNNS05SRHNiR1hlNWV4Unh4NWFrIiwiaWQiOiJ1cm46dXVpZDoyMjNlZDYzMS0yNzVjLTQzNTgtOWJhYS1jNmRiNTkzZjBjYzciLCJ0eXBlIjpbIlZlcmlmaWFibGVQcmVzZW50YXRpb24iXSwiQGNvbnRleHQiOlsiaH

Error: Invalid base64-encoded string: number of data characters (6781) cannot be 1 more than a multiple of 4